# EDA for NLP

In [1]:
import pandas as pd
from google.colab import drive
drive.mount("/google_drive")
WORKSPACE_PATH = "/google_drive/My Drive/MBD Capstone"

xls = pd.ExcelFile(WORKSPACE_PATH+ '/ie_input_data_no_character_sin_no.xlsx')
df = pd.read_excel(xls, 'Sheet1')

Mounted at /google_drive


In [2]:
len(df)

671

In [3]:
df_words = df[['text_answers','show_up']]
df_words = df['text_answers'].str.split(' ', expand=True)
df_words['label'] = df['show_up']
df_words = df_words.melt(id_vars='label')

In [4]:
df_words = df_words.drop(columns = 'variable')
df_words = df_words[~df_words['value'].isna()]
df_words['value'] = df_words['value'].str.lower()

In [5]:
import string
df_words['value'] = [s.translate(str.maketrans('', '', string.punctuation)) for s in df_words['value']]


In [6]:
# Stop words
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
  
example_sent = """This is a sample sentence,
                  showing off the stop words filtration."""
nltk.download("stopwords")
stop_words = set(stopwords.words('spanish'))
stop_words.remove('no')

df_words['stop_word']= [1 if val in stop_words else 0 for val in df_words['value']]
df_words = df_words[df_words['stop_word']!=1]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#Eliminate Numeric Values
df_words = df_words[~df_words['value'].str.contains('[0-9]')]

In [8]:
#Change accent
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a, b)
    return s

df_words['value'] = df_words['value'].apply(lambda x: normalize(x))

In [9]:
#Stemming
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')
df_words['word_stemmed'] = [spanish_stemmer.stem(word) for word in df_words['value']]

In [10]:
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp = es_core_news_sm.load()
#nlp = spc.load('es_core_news_sm')

def lemmatizer(text):  
  doc = nlp(text)
  return ' '.join([word.lemma_ for word in doc])

df_words['value_lemmatized'] = df_words['value'].apply(lambda x: lemmatizer(x))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 14.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [11]:
#Final Touches
df_words = df_words.rename(columns= {'label':'Show Up','value':'Words'})
df_words= df_words[df_words['Words']!='']
df_words

,Show Up,Words,stop_word,word_stemmed,value_lemmatized
0,1,si,0,si,si
1,1,si,0,si,si
2,0,si,0,si,si
3,1,si,0,si,si
4,1,si,0,si,si
...,...,...,...,...,...
85093,0,asistire,0,asistir,asistire
85764,0,gracias,0,graci,gracia
87106,0,lunes,0,lun,lunes
89119,0,abril,0,abril,abril


In [12]:
df_words.to_excel(WORKSPACE_PATH+ '/JLB/bag_of_words.xlsx')